In [99]:
import numpy as np
from astropy.io import fits

def create_synthetic_fits(file_path):
    # Create synthetic data for three bands
    red_band = np.random.normal(loc=100, scale=20, size=(100, 100))
    green_band = np.random.normal(loc=100, scale=20, size=(100, 100))
    blue_band = np.random.normal(loc=100, scale=20, size=(100, 100))

    # Create FITS HDUs
    hdu_red = fits.PrimaryHDU(red_band)
    hdu_green = fits.ImageHDU(green_band)
    hdu_blue = fits.ImageHDU(blue_band)

    # Create HDU list and write to file
    hdul = fits.HDUList([hdu_red, hdu_green, hdu_blue])
    hdul.writeto(file_path, overwrite=True)

if __name__ == '__main__':
    file_path = 'synthetic_data.fits'
    create_synthetic_fits(file_path)

In [101]:
import numpy as np
from astropy.io import fits
from PIL import Image, ImageEnhance
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

def calculate_snr(image_data):
    signal = np.mean(image_data)
    noise = np.std(image_data)
    return signal / noise

def enhance_image(image_data, enhancement_factor=1.5):
    # Apply Gaussian filter to smooth the image and reduce noise
    smoothed_image = gaussian_filter(image_data, sigma=2)
    image = Image.fromarray(smoothed_image)
    image = image.convert("L")
    enhancer = ImageEnhance.Contrast(image)
    enhanced_image = enhancer.enhance(enhancement_factor)
    return np.array(enhanced_image)

def combine_bands(red_band, green_band, blue_band):
    combined_image = np.stack((red_band, green_band, blue_band), axis=-1)
    return combined_image

def process_fits(file_path, enhancement_factor=1.5):
    with fits.open(file_path) as hdul:
        red_band = hdul[0].data
        green_band = hdul[1].data
        blue_band = hdul[2].data

        red_band_enhanced = enhance_image(red_band, enhancement_factor)
        green_band_enhanced = enhance_image(green_band, enhancement_factor)
        blue_band_enhanced = enhance_image(blue_band, enhancement_factor)

        combined_image = combine_bands(red_band_enhanced, green_band_enhanced, blue_band_enhanced)
        
        snr = calculate_snr(combined_image)
        if snr < 15:
            print(f"SNR is below 15: {snr}. Adjusting enhancement factor.")
            combined_image = combine_bands(
                enhance_image(red_band, enhancement_factor * 2),
                enhance_image(green_band, enhancement_factor * 2),
                enhance_image(blue_band, enhancement_factor * 2)
            )
        
        snr = calculate_snr(combined_image)
        print(f"Final SNR: {snr}")

if __name__ == '__main__':
    file_path = 'synthetic_data.fits'
    process_fits(file_path)

Final SNR: 23.056477991183353
